In [1]:
spark

In [6]:
superStoreDF = spark.read.csv("file:///C:/data/SuperStore/SuperStore_US_Sales.csv", header = True, inferSchema = True)
superStoreDF.printSchema()

root
 |-- Order_ID: string (nullable = true)
 |-- OrderDate: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub_Category: string (nullable = true)
 |-- Sales: double (nullable = true)
 |-- Quantitty: integer (nullable = true)
 |-- Region: string (nullable = true)



In [29]:
managerInfoDF = spark.read.csv("file:///C:/data/SuperStore/managerByRegion.csv", header = True, inferSchema = True)
managerInfoDF.printSchema()

root
 |-- Person: string (nullable = true)
 |-- Region: string (nullable = true)



In [30]:
returnInfoDF = spark.read.csv("file:///C:/data/SuperStore/totalReturns.csv", header = True, inferSchema = True).withColumnRenamed("Order_ID", "Returned_Order_ID")
returnInfoDF.printSchema()

root
 |-- Returned: string (nullable = true)
 |-- Returned_Order_ID: string (nullable = true)



In [21]:
joinExpression = superStoreDF["Region_1"] == managerInfoDF["Region"]

AnalysisException: 'Cannot resolve column name "Region_1" among (Order_ID, OrderDate, City, Category, Sub_Category, Sales, Quantitty, Region);'

In [40]:
superStoreDF1 = superStoreDF.withColumnRenamed("Region","Region_1")
superStoreDF1.cache()

superStoreDF1.join(managerInfoDF, superStoreDF1["Region_1"] == managerInfoDF["Region"]).columns

['Order_ID',
 'OrderDate',
 'City',
 'Category',
 'Sub_Category',
 'Sales',
 'Quantitty',
 'Region_1',
 'Person',
 'Region']

In [26]:
from pyspark.sql.functions import expr, col, column, desc, expr

superStoreDF1.join(managerInfoDF, superStoreDF1["Region_1"] == managerInfoDF["Region"] , 'leftouter').select('OrderDate', 'City', 'Sales','Region_1', col('Person').alias("manager") ).show()

+----------+-------------+-------+--------+-----------------+
| OrderDate|         City|  Sales|Region_1|          manager|
+----------+-------------+-------+--------+-----------------+
|10/19/2017|      Houston| 29.472| Central|   Kelly Williams|
| 12/9/2017|      Houston|  1.248| Central|   Kelly Williams|
| 12/9/2017|      Houston|  9.708| Central|   Kelly Williams|
| 12/9/2017|      Houston|  27.24| Central|   Kelly Williams|
|11/13/2017|      Chicago|230.376| Central|   Kelly Williams|
|10/26/2017|    Rochester|  19.99| Central|   Kelly Williams|
|10/26/2017|    Rochester|   6.16| Central|   Kelly Williams|
| 11/6/2017|     Portland|  5.682|    West|    Anna Andreadi|
| 11/9/2017|New York City|  96.53|    East|      Chuck Magee|
|11/23/2017|    Charlotte| 74.112|   South|Cassandra Brandow|
|11/23/2017|    Charlotte| 27.992|   South|Cassandra Brandow|
|11/23/2017|    Charlotte|  3.304|   South|Cassandra Brandow|
|12/25/2017|New York City|  41.96|    East|      Chuck Magee|
| 11/5/2

In [28]:
superStoreDF1.join(managerInfoDF, superStoreDF1["Region_1"] == managerInfoDF["Region"] , 'leftouter').select('OrderDate', 'City', 'Sales','Region_1', col('Person').alias("manager") ).explain()

== Physical Plan ==
*Project [OrderDate#122, City#123, Sales#126, Region_1#520, Person#155 AS manager#734]
+- *BroadcastHashJoin [Region_1#520], [Region#156], LeftOuter, BuildRight
   :- *Project [OrderDate#122, City#123, Sales#126, Region#128 AS Region_1#520]
   :  +- *FileScan csv [OrderDate#122,City#123,Sales#126,Region#128] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/C:/data/SuperStore/SuperStore_US_Sales.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<OrderDate:string,City:string,Sales:double,Region:string>
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[1, string, true]))
      +- *FileScan csv [Person#155,Region#156] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/C:/data/SuperStore/managerByRegion.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Person:string,Region:string>


In [46]:
""" identify the returned item """
join_expr_return = superStoreDF1["Order_ID"] == returnInfoDF["Returned_Order_ID"]

from pyspark.sql.functions import expr, col, column

returnINDSalesDF = superStoreDF1.join(returnInfoDF, join_expr_return, "leftOuter") #.where("Returned_Order_ID is not null")
returnINDSalesDF2 = returnINDSalesDF.select("*", expr("Returned_Order_ID is not null").alias("return_flag")).drop("Returned_Order_ID").drop("Returned")

returnINDSalesDF2.where("return_flag = true").show(4)

+--------------+----------+-----------+---------------+------------+-------+---------+--------+-----------+
|      Order_ID| OrderDate|       City|       Category|Sub_Category|  Sales|Quantitty|Region_1|return_flag|
+--------------+----------+-----------+---------------+------------+-------+---------+--------+-----------+
|CA-2017-137099| 12/7/2017|Los Angeles|     Technology|      Phones|374.376|        3|    West|       true|
|CA-2017-102519|11/27/2017|  Milwaukee|      Furniture| Furnishings|  46.94|        1| Central|       true|
|CA-2017-102519|11/27/2017|  Milwaukee|     Technology| Accessories| 143.73|        9| Central|       true|
|CA-2017-136539|12/28/2017| Round Rock|Office Supplies|         Art| 27.168|        2| Central|       true|
+--------------+----------+-----------+---------------+------------+-------+---------+--------+-----------+
only showing top 4 rows

